In [1]:
!pip install transformers
!pip install datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 12.6 MB/s 
     |████████████████████████████████| 163 kB 68.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 7.5 MB/s 
     |████████████████████████████████| 115 kB 17.4 MB/s 
     |████████████████████████████████| 212 kB 61.5 MB/s 
     |████████████████████████████████| 127 kB 56.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [3]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [4]:
text = "This is a great [MASK]."

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> This is a great deal.
>>> This is a great success.
>>> This is a great adventure.
>>> This is a great idea.
>>> This is a great feat.


In [8]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
import pandas as pd
%cd /content/drive/My Drive/Independent Study/
df = pd.read_parquet('all_2020_US_Election_News.parquet', engine='pyarrow')

# LOADING DATASET IN HUGGING FACE FORM
# from datasets import load_dataset
# df = load_dataset("parquet", data_files={'all_2020_US_Election_News.parquet'})

/content/drive/My Drive/Independent Study


In [10]:
df

,Unnamed: 0,id,author,date,title,publication,text,keywords,summary
0,0,1,The Associated Press,2020-03-23 19:19:00,Review: 'The Glass Hotel a tragic tale for the...,ABC News,"The Glass Hotel, Alfred A. Knopf, by Emily St....","['hotel', 'tragic', 'vancouver', 'end', 'revie...","The Glass Hotel, Alfred A. Knopf, by Emily St...."
1,1,2,COLLEEN LONG and MARIA VERZA Associated Press,2020-03-23 11:21:34,Malaria drugs' promise for coronavirus spurs h...,ABC News,Malaria drugs' promise for coronavirus spurs h...,"['drug', 'virus', 'spurs', 'malaria', 'health'...",Malaria drugs' promise for coronavirus spurs h...
2,2,3,Ben Gittleson,2020-03-23 18:37:34,Malaria drugs' promise for coronavirus spurs h...,ABC News,Malaria drugs' promise for coronavirus spurs h...,"['drug', 'virus', 'spurs', 'malaria', 'health'...",Malaria drugs' promise for coronavirus spurs h...
3,3,4,Libby Cathey,2020-03-23 18:37:37,Senate showdown over pandemic relief stalls as...,ABC News,Senate showdown over pandemic relief stalls as...,"['relief', 'democrats', 'pelosi', 'vote', 'lea...",Senate showdown over pandemic relief stalls as...
4,4,5,ABC News,2020-03-23 19:52:06,Fallout of '08 bailout looms over Washington n...,ABC News,Fallout of '08 bailout looms over Washington n...,"['financial', 'money', '08', 'street', 'wall',...",On the table: a nearly $2 trillion rescue pack...
...,...,...,...,...,...,...,...,...,...
144367,144367,144714,"USA TODAY, Aimee Picchi, Special to USA TODAY",2021-01-21 08:31:10,'Andy spread election fraud lies': Arizona Rep...,USA Today,U.S. Rep. Andy Biggs two brothers have publicl...,"['election', 'lies', 'spread', 'brothers', 'me...",U.S. Rep. Andy Biggs two brothers have publicl...
144368,144368,144715,"Reviewed, Rachel Dube, Reviewed.com",2021-01-21 18:59:35,Amazon offers to help Biden administration wit...,USA Today,Amazon is offering its help to President Joe B...,"['vaccine', 'administration', 'help', 'letter'...",Amazon is offering its help to President Joe B...
144369,144369,144716,"Reviewed, Courtney Campbell, Reviewed",2021-01-21 20:35:05,Taron Johnson got out of the doghouse when Buf...,USA Today,The last time the Buffalo Bills played the Kan...,"['bills', 'didnt', 'play', 'played', 'buffalo'...",The last time the Buffalo Bills played the Kan...
144370,144370,144717,"USA TODAY, Charisse Jones and Matt Wynn, USA T...",2021-01-21 05:01:02,President Biden and Vice President Harris visi...,USA Today,President Biden and Vice President Harris visi...,"['tomb', 'vice', 'unknown', 'wreathlaying', 'c...",President Biden and Vice President Harris visi...


In [11]:
# for index, row in df.iterrows():
#   if row.publication == 'ABC News':
#     row['leaning'] = 'L'
#   break
def label_leaning(row):
  if row['publication'] == 'ABC News' or row['publication'] == 'CBS News' or row['publication'] == 'CNN' or row['publication'] == 'MSNBC' or row['publication'] == 'The Washington Post' or row['publication'] == 'Al Jazeera English' or row['publication'] == 'Google News' or row['publication'] == 'Buzzfeed':
    return 'L'
  elif row['publication'] in ['Fox News', 'Breitbart News', 'The Washington Times']:
    return 'R'
  return 'C'

df['leaning'] = df.apply (lambda row: label_leaning(row), axis=1)


In [12]:
df.iloc[0]

Unnamed: 0                                                     0
id                                                             1
author                                      The Associated Press
date                                         2020-03-23 19:19:00
title          Review: 'The Glass Hotel a tragic tale for the...
publication                                             ABC News
text           The Glass Hotel, Alfred A. Knopf, by Emily St....
keywords       ['hotel', 'tragic', 'vancouver', 'end', 'revie...
summary        The Glass Hotel, Alfred A. Knopf, by Emily St....
leaning                                                        L
Name: 0, dtype: object

In [13]:
print(df.size)

1443720


In [14]:
df2 = pd.DataFrame().assign(text=df['text'], leaning=df['leaning'])
df2 = df2.dropna()

df2.size

277754

In [15]:
grouped_df = df2.groupby('leaning')

In [16]:
grouped_df = df2.groupby('leaning').apply(lambda a: a[:])

In [17]:
dfs = [x for _, x in df2.groupby('leaning')]

In [18]:
C_dataset, L_dataset, R_dataset = dfs[0], dfs[1], dfs[2]

In [19]:
C_dataset.to_csv (r'export_C_dataframe.csv', index = False, header=True)
L_dataset.to_csv (r'export_L_dataframe.csv', index = False, header=True)
R_dataset.to_csv (r'export_R_dataframe.csv', index = False, header=True)

In [20]:
# LOADING L DATASET IN HUGGING FACE FORM
from datasets import load_dataset
%cd /content/drive/My Drive/Independent Study/

dataset = load_dataset("csv", data_files={'export_R_dataframe.csv'})

/content/drive/My Drive/Independent Study


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b127eee6a4968bfb/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
dataset = dataset['train']

In [22]:
train = dataset.shuffle(seed=42).select(range(20000))
test = dataset.shuffle(seed=42).select(range(20000, 40000))
unsupervised = dataset.shuffle(seed=42).select(range(40000, len(dataset)))

In [23]:
# pip install datasets
my_dataset_dict = {
    "train": train,
    "test": test,
    "unsupervised": unsupervised
    }

In [24]:
len(my_dataset_dict['unsupervised'])

3271

In [25]:
my_dataset_dict

{'train': Dataset({
     features: ['text', 'leaning'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['text', 'leaning'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['text', 'leaning'],
     num_rows: 3271
 })}

In [26]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# my_dataset_dict = my_dataset_dict.dropna()
# my_dataset_dict = my_dataset_dict['train']['tr'].tolist()
# texts_encodings = tokenizer(texts, truncation=True, padding=True)

# Use batched=True to activate fast multithreading!
train_tokenized_datasets = my_dataset_dict['train'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
test_tokenized_datasets = my_dataset_dict['test'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
unsupervised_tokenized_datasets = my_dataset_dict['unsupervised'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)

  0%|          | 0/20 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [27]:
print(len(train_tokenized_datasets), len(test_tokenized_datasets), len(unsupervised_tokenized_datasets))

20000 20000 3271


In [28]:
tokenized_dataset = {
     "train": train_tokenized_datasets,
    "test": test_tokenized_datasets,
    "unsupervised": unsupervised_tokenized_datasets
}

In [29]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 3271
 })}

In [30]:
tokenizer.model_max_length
chunk_size = 128


In [31]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 320'
'>>> Review 1 length: 851'
'>>> Review 2 length: 580'


In [32]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 1751'


In [33]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 87'


In [34]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [35]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 3271
 })}

In [36]:
lm_train_datasets = tokenized_dataset['train'].map(group_texts, batched=True)
lm_test_datasets = tokenized_dataset['test'].map(group_texts, batched=True)
lm_unsupervised_datasets = tokenized_dataset['unsupervised'].map(group_texts, batched=True)

lm_datasets = {
     "train": lm_train_datasets,
    "test": lm_test_datasets,
    "unsupervised": lm_unsupervised_datasets
}

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [37]:
lm_datasets

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 110389
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 110933
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 18083
 })}

In [38]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])


'sleepy joe biden just closed down his campaign for the day ( again ). wants to rest! he is a very low energy individual, and our country cannot make it in these exciting, but complex and competitive times, with a low energy president!!! donald j. trump ( @ realdonaldtrump ) september 24, 2020 trump is revisiting the insult that he used against former florida gov. jeb bush ( r ) during the 2016 republican presidential primary. i used to call jeb bush, low energy. but i mean, if you compare the two, jeb is extraordinarily energetic compared to sleepy'

In [39]:
tokenizer.decode(lm_datasets["train"][1]["labels"])


'sleepy joe biden just closed down his campaign for the day ( again ). wants to rest! he is a very low energy individual, and our country cannot make it in these exciting, but complex and competitive times, with a low energy president!!! donald j. trump ( @ realdonaldtrump ) september 24, 2020 trump is revisiting the insult that he used against former florida gov. jeb bush ( r ) during the 2016 republican presidential primary. i used to call jeb bush, low energy. but i mean, if you compare the two, jeb is extraordinarily energetic compared to sleepy'

# **Fine-tuning DistilBERT with the Trainer API**


In [40]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [41]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] president [MASK] trump mocked his rival former vice president joe biden for hosting zero public [MASK] on thursday. sleepy joe biden just closed down his campaign for the day ( again ), trump wrote on twitter. wants to rest! the bid [MASK] campaign called a lid for reporters awaiting any activity [MASK] the campaign on thursday, as the former vice president is expected to focus on preparation for the debate. trump mocked [MASK]en for his lack of energy on the campaign trail. he is a very low energy individual, and our country cannot make it in these exciting, but complex andbridge times, with [MASK] [MASK] energy president!!! he wrote.'

'>>> sleepy joe biden [MASK] closed down hesse [MASK] for [MASK] day ( [MASK] [MASK]. [MASK] to [MASK] [MASK] he [MASK] a very [MASK] energy individual, and our country cannot make it in these exciting [MASK] but [MASK] and hopkins times, with a [MASK] energy president!!! donald j. trump ( @ real [MASK]aldtrum [MASK] ) [MASK] 24, [MASK] tru

In [42]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return tf_default_data_collator(features)

In [43]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] president [MASK] trump mocked his rival former vice president joe [MASK] [MASK] for hosting zero [MASK] events on thursday. sleepy joe biden [MASK] closed down his campaign for the day ( again ), trump wrote on twitter. wants to rest! the [MASK] [MASK] campaign [MASK] a [MASK] [MASK] reporters awaiting any activity [MASK] [MASK] campaign on thursday, as [MASK] [MASK] [MASK] president [MASK] expected to focus on preparation [MASK] the debate. trump mocked biden for [MASK] lack of energy on the [MASK] [MASK]. he is a very low energy individual [MASK] and our [MASK] cannot make it in these exciting, but complex and competitive [MASK] [MASK] with a low [MASK] president!!! he wrote.'

'>>> sleepy joe biden just closed down his campaign for the day ( again ). [MASK] [MASK] [MASK] [MASK] he is a very low [MASK] individual, and [MASK] country [MASK] [MASK] it in these exciting, but [MASK] and competitive times, with a [MASK] energy [MASK]!!! donald j. trump ( [MASK] realdonaldtrump

In [44]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [45]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token


In [48]:
tf_train_dataset = downsampled_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = downsampled_dataset["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)


In [49]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

callback = PushToHubCallback(
    output_dir=f"R-Leaning-finetuned-BERT", tokenizer=tokenizer
)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/rhsarvai/R-Leaning-finetuned-BERT into local empty directory.


In [50]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 11s 209ms/step - loss: 3.2079
Perplexity: 24.73


In [51]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

  6/313 [..............................] - ETA: 3:07 - loss: 3.3197

313/313 [==============================] - 222s 671ms/step - loss: 2.9303 - val_loss: 2.6097


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


KeyboardInterrupt: ignored

In [52]:
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [53]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 7s 223ms/step - loss: 2.6013
Perplexity: 13.48


In [54]:
# Saving the model
model.save_weights('./checkpoints/my_r_leaning_BERT')

## Using the Fine-tuned model

In [55]:
!pip install transformers

# Create a new model instance
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [56]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/

# Restore the weights
model.load_weights('./checkpoints/my_r_leaning_BERT')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Independent Study


In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="R-Leaning-finetuned-BERT"
)

In [ ]:
prediction_sentence = 'Trump considering [MASK] coronavirus guidelines amid growing economic fallout'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> trump considering the coronavirus guidelines amid growing economic fallout
>>> trump considering expanding coronavirus guidelines amid growing economic fallout
>>> trump considering implementing coronavirus guidelines amid growing economic fallout
>>> trump considering new coronavirus guidelines amid growing economic fallout
>>> trump considering adopting coronavirus guidelines amid growing economic fallout


In [ ]:
prediction_sentence = 'Biden considering [MASK] coronavirus guidelines amid growing economic fallout'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> biden considering the coronavirus guidelines amid growing economic fallout
>>> biden considering implementing coronavirus guidelines amid growing economic fallout
>>> biden considering new coronavirus guidelines amid growing economic fallout
>>> biden considering adopting coronavirus guidelines amid growing economic fallout
>>> biden considering changing coronavirus guidelines amid growing economic fallout


In [ ]:
# model.push_to_hub("my-r-leaning-finetuned-bert")

In [ ]:
validation_sentences_df = pd.read_csv('PoliticalBiasValidationSentences2.csv')
print(validation_sentences_df['Masked Sentences'])

0     Trump [MASK] the sad news on Monday in the pre...
1     Trump [MASK] the elections after the results y...
2            Biden [MASK] the victory of Obama in 2012.
3              Trump was [MASK] after the win of Obama.
4     Trump [MASK] tweets as US tackles coronavirus,...
5     Trump considering [MASK] coronavirus guideline...
6     Biden considering [MASK] coronavirus guideline...
7     Trump intends to [MASK] country amid growing c...
8     Biden says he intendes to [MASK] country to pr...
9     New York mayor [MASK] trump for medical supply...
10    New York mayor [MASK] Biden irrespective of el...
11    Biden says it's time for Trump to [MASK] and m...
12    Biden [MASK] Trump after republicans block cov...
13    Trump [MASK] to restart the US economy because...
14    Biden [MASK] new bill specially directed towar...
15    Trump administration has shown [MASK] interest...
16    Biden administration has shown [MASK] interest...
17    Trump considering [MASK] coronavirus guide

In [ ]:
for prediction in validation_sentences_df['Masked Sentences']:
  preds = mask_filler(prediction)
  for pred in preds:
    print(f">>> {pred['sequence']}")
  print()


>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump covered the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump suspended the elections after the results yesterday.

>>> biden predicted the victory of obama in 2012.
>>> biden supported the victory of obama in 2012.
>>> biden endorsed the victory of obama in 2012.
>>> biden praised the victory of obama in 2012.
>>> biden supports the victory of obama in 2012.

>>> trump was reelected after the win of obama.
>>> trump was elected after the win of obama.
>>> trump was fired after the

# Validation

In [ ]:
import pandas as pd
validation_sentences_df = pd.read_csv('PoliticalBiasValidationSentences.csv')
print(validation_sentences_df['Masked Sentences'])

In [ ]:
top_most_scores = []
top_most_predictions = []
top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = mask_filler(prediction)
  print(preds[0])
  top_most_scores.append(preds[0]['score'])
  top_most_predictions.append(preds[0]['token_str'])
  top_most_sentences.append(preds[0]['sequence'])

  # print(preds[0])
  for pred in preds:
    print(f">>> {pred['sequence']}")
    # print(pred)
  print()

In [ ]:
print(top_most_predictions[2], top_most_sentences[2])

# **Scoring Validation**

In [ ]:
lexicons = pd.read_csv('NRC-VAD-Lexicon.csv')
lexicons = lexicons[lexicons['English Word'].notnull()]
print(lexicons)

In [ ]:
# Lemmatizing word before it's put in the dictionary
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer